In [ ]:
%pip install -r requirements.txt

In [2]:
%reload_ext dotenv
%dotenv .env

In [4]:
from langchain.text_splitter import Language
from langchain.document_loaders.generic import GenericLoader
from langchain.document_loaders.parsers import LanguageParser

repo_path = '/Users/lavesh/Documents/Hakbah/Nest_API/src'

loader = GenericLoader.from_filesystem(
    repo_path,
    glob="**/*",
    suffixes=[".ts"],
    parser=LanguageParser(language=Language.JS),
    show_progress=True,
)
raw_code = loader.load()


/Users/lavesh/anaconda3/envs/invoicing/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
100%|██████████| 637/637 [00:00<00:00, 1782.45it/s]


In [21]:
from langchain.text_splitter import (
    RecursiveCharacterTextSplitter,
    Language,
)

ts_splitter = RecursiveCharacterTextSplitter.from_language(
    language=Language.TS, chunk_size=1000, chunk_overlap=100
)
ts_docs = ts_splitter.split_documents(raw_code)

In [22]:
from langchain.embeddings import SentenceTransformerEmbeddings

embeddings = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")

In [23]:
from langchain.vectorstores import Chroma

db = Chroma.from_documents(ts_docs, embeddings)

In [24]:
retriever = db.as_retriever(
    search_type="mmr",
    search_kwargs={"k": 8},
    # search_type="similarity",
)

In [25]:
from langchain.chat_models import ChatOpenAI
from langchain.memory import ConversationSummaryMemory
from langchain.chains import ConversationalRetrievalChain

llm = ChatOpenAI(model_name="gpt-3.5-turbo")
memory = ConversationSummaryMemory(
    llm=llm, memory_key="chat_history", return_messages=True
)
qa = ConversationalRetrievalChain.from_llm(llm, retriever=retriever, memory=memory)

In [26]:
import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"

In [27]:
questions = [
    "What are the conditions to trigger an payout?"
]

for question in questions:
    result = qa(question)
    print(f"-> *Question*: {question} \n")
    print(f"*Answer*: {result['answer']} \n")

-> *Question*: What are the conditions to trigger an payout? 

*Answer*: To trigger a payout, the following conditions need to be met:

1. The payout date for the cycle must have been reached.
2. If the payout requires the acceptance of a sanad, the sanad must be accepted.
3. If it is a cashback payment, the user must have all previous contributions paid.
4. The cashback value must not be negative.
5. The cashback value for the cycle must be greater than 0.
6. The target jameya must not have started or been deleted/dismissed in order to modify its cycle cashback property.
7. The target jameya must be in a complete status to claim the cashback payment.
8. The user must be eligible for a cashback payment, which means they have made all the required contributions and do not have any late contributions. 

